In [4]:
import os
import re
import reactionmodel.load
from typing import NamedTuple
import numpy as np

os.chdir('/Users/thayer/develop/reactionmodel/')
root = '/Users/thayer/develop/hybrid//tests/sbml-tests/sbml-003-dimerisation/'

#root = './tests/sbml-tests/sbml-001-birth-death/'
check = 'p01i01'

def get_path_from_check_string(check_string, prefix, directory_name, filename):
    match = re.search(f'{prefix}([0-9]+)', check_string)
    model_path = f'{directory_name}/{filename}{match[1]}.txt' if match is not None else f'{filename}.txt'
    return model_path

class SpecificationPaths(NamedTuple):
    model_path: str
    parameters_path: str
    config_path: str
    initial_condition_path: str

def get_path_tuple(root, check):
    model_path = os.path.join(root, get_path_from_check_string(check, 'm', 'models', 'model'))
    parameters_path = os.path.join(root, get_path_from_check_string(check, 'p', 'parameters', 'parameters'))
    config_path = os.path.join(root, get_path_from_check_string(check, 'c', 'configs', 'config'))
    ic_path = os.path.join(root, get_path_from_check_string(check, 'i', 'initial_conditions', 'initial'))

    return SpecificationPaths(model_path, parameters_path, config_path, ic_path)

In [5]:
specification = reactionmodel.load.load_specification(*get_path_tuple(root, check))
specification.simulation_options

{'partition': '/Users/thayer/develop/hybrid//tests/sbml-tests/sbml-003-dimerisation/./partition.json'}

In [6]:
prop = specification.model.get_propensities_function(jit=False, parameters=specification.parameters)
prop(0, np.array([100.0, 0.]))

Evaluating expression: k1 => 0.001
Evaluating expression: k2 => 0.01


array([4.95, 0.  ])

In [7]:
prop = specification.model.get_propensities_function(jit=True, parameters=specification.parameters)
prop(0, np.array([100.0, 0.]))

Evaluating expression: k1 => 0.001
Evaluating expression: k2 => 0.01


array([4.95, 0.  ])

In [8]:
prop = specification.model._get_jit_propensities_function(parameters=specification.parameters)
prop(0, np.array([100.0, 0.]))

Evaluating expression: k1 => 0.001
Evaluating expression: k2 => 0.01


array([4.95, 0.  ])

In [9]:
dydt = specification.model.get_dydt(parameters=specification.parameters)
dydt(0, np.array([100.0, 0.]))

Evaluating expression: k1 => 0.001
Evaluating expression: k2 => 0.01


array([-9.9 ,  4.95])

In [10]:
dydt = specification.model.get_dydt(jit=True, parameters=specification.parameters)
dydt(0, np.array([100.0, 0.]))

Evaluating expression: k1 => 0.001
Evaluating expression: k2 => 0.01


array([-9.9 ,  4.95])

In [11]:
from dataclasses import asdict
import json

from reactionmodel.model import Species, Reaction, ReactionRateFamily, Model

In [12]:
a = Species('A')
r1 = Reaction('r1', a, (a, 2))
r2 = Reaction('r2', a, (a, 2))

def k_of_t(t):
    return t**2

fam = ReactionRateFamily([r1,r2], k_of_t)

m = Model(a, fam)

In [13]:
len(fam.reactions)

2

In [14]:
{'k_of_t': k_of_t}

{'k_of_t': <function __main__.k_of_t(t)>}

In [15]:
str(k_of_t.__name__)

'k_of_t'

In [16]:
k_of_t.__name__

'k_of_t'

In [17]:
Model.from_dict(m.to_dict(), functions_by_name={'k_of_t': k_of_t})

In [18]:
m.to_dict()

{'species': [{'name': 'A', 'description': ''}],
 'reaction_groups': [{'reactions': [{'name': 'r1',
     'reactants': ({'name': 'A', 'description': ''},),
     'products': (({'name': 'A', 'description': ''}, 2),),
     'description': '',
     'kinetic_orders': ({'name': 'A', 'description': ''},),
     'reversible': False,
     'k': None},
    {'name': 'r2',
     'reactants': ({'name': 'A', 'description': ''},),
     'products': (({'name': 'A', 'description': ''}, 2),),
     'description': '',
     'kinetic_orders': ({'name': 'A', 'description': ''},),
     'reversible': False,
     'k': None}],
   'k': 'k_of_t'}]}